In [1]:
from ib_insync import *
import pandas as pd
from IPython.display import display, clear_output
util.startLoop()
ib = IB()

In [2]:
# ib = IB()
ib.connect('localhost', 7497, clientId=2)

<IB connected to localhost:7497 clientId=2>

In [3]:
contracts = [Forex(pair) for pair in ("EURUSD", "USDJPY", "GBPUSD")]
ib.qualifyContracts(*contracts)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'),
 Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY'),
 Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')]

In [4]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]
eurusd

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

In [5]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

In [6]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 5, 13, 2, 52, 34, 406051, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.07706, bidSize=1000000.0, ask=1.07707, askSize=3000000.0, prevBidSize=2500000.0, prevAsk=1.07708, prevAskSize=2000000.0, high=1.07755, low=1.0766, close=1.0771)

In [7]:
df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
        display(df)        


# ib.sleep(30)
# ib.pendingTickersEvent -= onPendingTickers

In [ ]:
ib.pendingTickersEvent += onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,2000000.0,1.07717,1.07719,2000000.0,1.07755,1.0766,1.0771
USDJPY,2000000.0,155.718,155.722,1000000.0,155.96,155.505,155.78
GBPUSD,1000000.0,1.25253,1.25256,1500000.0,1.25275,1.25175,1.2525
USDCHF,1000000.0,0.90647,0.90653,2000000.0,0.9077,0.90565,0.90645
USDCAD,1000000.0,1.36806,1.36811,1000000.0,1.369,1.3669,1.3673
AUDUSD,8000000.0,0.6598,0.65983,2000000.0,0.66065,0.65855,0.6603


In [9]:
ib.pendingTickersEvent -= onPendingTickers

In [20]:
bars = ib.reqHistoricalData(
    contract=eurusd,
    endDateTime="",
    durationStr="60 S",
    barSizeSetting="10 secs",
    whatToShow="MIDPOINT",
    useRTH=False,
    formatDate=True,
    keepUpToDate=True
)

In [29]:
pd.DataFrame(bars)

,date,open,high,low,close,volume,average,barCount
0,2024-05-12 22:59:30-04:00,1.077150,1.077150,1.077150,1.077150,-1.0,-1.0,-1
1,2024-05-12 22:59:40-04:00,1.077150,1.077150,1.077125,1.077140,-1.0,-1.0,-1
2,2024-05-12 22:59:50-04:00,1.077140,1.077140,1.077125,1.077130,-1.0,-1.0,-1
3,2024-05-12 23:00:00-04:00,1.077130,1.077130,1.077105,1.077125,-1.0,-1.0,-1
4,2024-05-12 23:00:10-04:00,1.077125,1.077125,1.077105,1.077110,-1.0,-1.0,-1
5,2024-05-12 23:00:20-04:00,1.077110,1.077110,1.077105,1.077105,-1.0,-1.0,-1
6,2024-05-12 23:00:30-04:00,1.077110,1.077110,1.077090,1.077105,-1.0,-1.0,-1
7,2024-05-12 23:00:40-04:00,1.077105,1.077105,1.077030,1.077030,-1.0,-1.0,-1
8,2024-05-12 23:00:50-04:00,1.077030,1.077045,1.077030,1.077030,-1.0,-1.0,-1
9,2024-05-12 23:01:00-04:00,1.077030,1.077060,1.077030,1.077060,-1.0,-1.0,-1


In [ ]:
ib.cancelHistoricalData(bars)

Error 162, reqId 23: Historical Market Data Service error message:API historical data query cancelled: 23


In [30]:
ib.disconnect()